# BIG DATA Y COMPUTACIÓN DE ALTO DESEMPEÑO: UNA INTRODUCCIÓN PARA EL ESTADÍSTICO



Haga click [aquí](http://nbviewer.jupyter.org/format/slides/github/jdvelasq/big-data-y-comp-alto-desempeno/blob/master/Slides.ipynb?flush_cache=true#/) para ver esta presentación en nbviewer.

<img src="img/firma-social-media.jpg" alt="" width="500" >

<img src="img/evo-big-data.jpg" alt="" width="800" >

<img src="img/servicios-web.jpg" alt="" width="800" >

# Apache Hadoop & Map/Reduce

<img src="img/map-reduce.jpg" alt="" width="800" >

<img src="img/mr-word-count.jpg" alt="" width="800" >

# Generalidades

## Modos de ejecución


* Modo local (un solo hilo de ejecución en la máquina local, para aprendizaje y desarrollo). El sistema local de archivos funciona como el HDFS.


* `=====>`  Modo seudo-distribuido (múltiples hilos en la máquina local, para desarrollo). El HDFS es alcanzable como un servicio.


* Cluster (ambiente productivo)



## Desarrollo de aplicaciones


* Programación directa en Java (Hadoop)


* `=====>` Programación usando un lenguaje interpretado (Hadoop-streaming)



# Sistemas operativos

* Microsoft Windows (dificil).


* Mac OS: instalación directa. 


* Ubuntu Linux: instalación directa.



# Ejemplo del conteo de palabras

Se usará Hadoop-streaming.

## Datos

In [93]:
%%writefile input/text0.txt
A B C A
A D D A
A K M C
 

Overwriting input/text0.txt


In [94]:
%%writefile input/text1.txt
B A C Y B
U O Y Y A
A B I T
 

Overwriting input/text1.txt


In [95]:
%%writefile input/text2.txt
A C D A
A K B 
A N H I D A
 

Overwriting input/text2.txt


## mapper.R

In [96]:
%%writefile mapper.R
#! /usr/bin/env Rscript

input <- file('stdin', 'r')
while(TRUE) {
    row <- readLines(input, n=1)
    if( length(row) == 0 ){
        break
    }
    words <- strsplit(row, " ")[[1]]
    for(word in words){
        if(word != '')
            write(cat(word,'\t1',sep=''), "")
    }
}

Overwriting mapper.R


In [119]:
## El programa anterior se hace ejecutable
!chmod +x mapper.R

## Verificación
!cat ./input/text*.txt | ./mapper.R | head

A	1
B	1
C	1
A	1
A	1
D	1
D	1
A	1
A	1
K	1
Error in cat(x, file = file, sep = c(rep.int(sep, ncolumns - 1), "\n"),  : 
  ignoring SIGPIPE signal
Calls: write -> cat
Execution halted


## reducer.R

In [98]:
%%writefile reducer.R
#!/usr/bin/env Rscript

curkey <- NULL
total <- 0
input <- file('stdin', 'r')
while(TRUE) {
    row <- readLines(input, n=1)
    if( length(row) == 0 ){
        break
    }
    x <- strsplit(row, "\t")[[1]]
    key <- x[1]
    value <- strtoi(x[2])
    if(!is.null(curkey) && key == curkey){
        total <- total + value
    }
    else{
        if( !is.null(curkey) ) {
            write(cat(curkey,'\t', total), "")
        }
        curkey <- key
        total <- value
    }
}
write(cat(curkey,'\t', total), "")

Overwriting reducer.R


In [99]:
!chmod +x reducer.R
!cat ./input/text*.txt | ./mapper.R | sort | ./reducer.R 

Warning message:
In readLines(input, n = 1) : incomplete final line found on 'stdin'
A 	 13
B 	 5
C 	 4
D 	 4
H 	 1
I 	 2
K 	 2
M 	 1
N 	 1
O 	 1
T 	 1
U 	 1
Y 	 3


## Ejecución en Hadoop

In [115]:
!hadoop fs -mkdir /user
!hadoop fs -mkdir /user/jdvelasq
!hadoop fs -mkdir /user/jdvelasq/input
!hadoop fs -copyFromLocal  input/* /user/jdvelasq/input
!hadoop fs -ls /user/jdvelasq/input/*

-rw-r--r--   1 jdvelasq supergroup         25 2018-11-08 23:36 /user/jdvelasq/input/text0.txt
-rw-r--r--   1 jdvelasq supergroup         29 2018-11-08 23:36 /user/jdvelasq/input/text1.txt
-rw-r--r--   1 jdvelasq supergroup         28 2018-11-08 23:36 /user/jdvelasq/input/text2.txt


In [116]:
!hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar \
-input input -output output  -mapper mapper.R -reducer reducer.R

In [117]:
!hadoop fs -ls /user/jdvelasq/output

Found 2 items
-rw-r--r--   1 jdvelasq supergroup          0 2018-11-08 23:37 /user/jdvelasq/output/_SUCCESS
-rw-r--r--   1 jdvelasq supergroup         79 2018-11-08 23:37 /user/jdvelasq/output/part-00000


In [118]:
!hadoop fs -cat /user/jdvelasq/output/part-00000

A 	 13
B 	 5
C 	 4
D 	 4
H 	 1
I 	 2
K 	 2
M 	 1
N 	 1
O 	 1
T 	 1
U 	 1
Y 	 3


## Problemas Típicos que se puden programar usando Map/Reduce

* Valores máximo o mínimo para una clave.


* Ordenamiento del archivo.


* Sumas y promedios por claves.


* Obtener los N registros más ...


* Asociar por claves:


    0   A, B                  A  0, 1
    1   A, B       ======>    B  0, 1, 2  
    2   B, C                  C  2
    
    
**La mayor parte de problemas no pueden solucionarse usando un proceso simple Map/Reduce**

<img src="img/mr-jobs.jpg" alt="" width="800" >

# Apache Pig

<img src="img/big-data-analytics.jpg" alt="" width="800" >

----

In [ ]:
from IPython.core.display import HTML
def css():
    style = open("custom.css", "r").read()
    return HTML(style)
css()

---